# Visualising results of cell-cell communication analysis in NanoString GeoMx® human kidney dataset

#### Interactions that are dysregulated in abnormal glomeruli (cancerous) vs healthy glomeruli

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scanpy as sc
import pyvis
from pyvis.network import Network
import networkx as nx

sc.settings.verbosity = 3  # verbosity: errors (0), warnings (1), info (2), hints (3)
#sc.logging.print_versions()
sc.settings.set_figure_params(dpi=80)  # low dpi (dots per inch) yields small inline figures

### Reading in results of cell-cell communication analysis with CellPhoneDB

In [2]:
interactions_upreg = pd.read_csv('./cellphone_interactions_table_with_gene_stats_downreg_in_abnormal_vs_healthy_no_logFC_cutoff_all.csv',
                                index_col=0)
interactions_downreg = pd.read_csv('./cellphone_interactions_table_with_gene_stats_upreg_in_abnormal_vs_healthy_no_logFC_cutoff_all.csv',
                                index_col=0)

In [3]:
# declare the cell types we are working with in our datasets
cell_types = ['B cell','CD8 T cell','DC','Distal_tubules_and_collecting_duct',
              'Endothelium','Loop_of_Henle','MNP','Mast cell','Mesangial_cells',
              'NK cell','NKT cell','Nephron_epithelium','Neutrophil','Podocytes',
              'Private','Proximal_tubular_cells','Th cell']

#### Read in spatially colocated cell types (from S5 notebook / step 6 in the workflow)

In [4]:
pairs_coloc_df = pd.read_csv('./coloc_cell_types.csv', index_col=0)

In [5]:
pairs_coloc = []

for idx in pairs_coloc_df.index:
    pairs_coloc.append(tuple(pairs_coloc_df.loc[idx]))

In [6]:
len(pairs_coloc)

104

In [7]:
# adding self pairs
pairs_coloc_all = pairs_coloc + [(elem, elem) for elem in cell_types]
len(pairs_coloc_all)

121

In [8]:
pairs_coloc_all

[('B cell', 'Mesangial_cells'),
 ('B cell', 'MNP'),
 ('Mesangial_cells', 'Nephron_epithelium'),
 ('Loop_of_Henle', 'Mast cell'),
 ('Distal_tubules_and_collecting_duct', 'NK cell'),
 ('MNP', 'Podocytes'),
 ('Loop_of_Henle', 'Nephron_epithelium'),
 ('DC', 'Podocytes'),
 ('Distal_tubules_and_collecting_duct', 'Nephron_epithelium'),
 ('MNP', 'Private'),
 ('Endothelium', 'Mesangial_cells'),
 ('Endothelium', 'MNP'),
 ('DC', 'Private'),
 ('B cell', 'CD8 T cell'),
 ('B cell', 'Loop_of_Henle'),
 ('B cell', 'DC'),
 ('B cell', 'Nephron_epithelium'),
 ('CD8 T cell', 'Endothelium'),
 ('Mast cell', 'Private'),
 ('Loop_of_Henle', 'Neutrophil'),
 ('NK cell', 'Neutrophil'),
 ('DC', 'MNP'),
 ('Distal_tubules_and_collecting_duct', 'Mesangial_cells'),
 ('Nephron_epithelium', 'Neutrophil'),
 ('Endothelium', 'Loop_of_Henle'),
 ('B cell', 'Neutrophil'),
 ('Endothelium', 'Nephron_epithelium'),
 ('MNP', 'NK cell'),
 ('DC', 'NK cell'),
 ('MNP', 'Proximal_tubular_cells'),
 ('Distal_tubules_and_collecting_duct', 

### Producing tables of up / downregulated interactions for each pair of spatially colocated cell types

### Explore your favourite cell types talking to each other

To so that, simply have a look at the `pairs_coloc_all` list (as shown above) and pass a particular cell type pair into either `interactions_upreg_pair` or `interactions_downreg_pair` dictionaries to see the interactions dysregulated in abnormal glomeruli

In [9]:
# subset tables for each cell type pair, including self interactions

interactions_upreg_pair = {}

for pair in pairs_coloc_all:
    ct1 = pair[0]
    ct2 = pair[1]
    interactions_upreg_pair[pair] = interactions_upreg[((interactions_upreg['celltype_A'] == ct1) & (interactions_upreg['celltype_B'] == ct2))
                                                | ((interactions_upreg['celltype_B'] == ct1) & (interactions_upreg['celltype_A'] == ct2))]


In [10]:
interactions_upreg_pair[('B cell','Mesangial_cells')]

,interaction,partner_A_genes,partner_B_genes,celltype_A,celltype_B,is_partner_A_DE,logFC_gene_A,adj_pval_gene_A,percent_expr_gene_A,is_partner_B_DE,logFC_gene_B,adj_pval_gene_B,percent_expr_gene_B
300,CD74_APP,CD74,APP,B cell,Mesangial_cells,True,-52.107051,1.434900e-02,1,False,NaN,NaN,0.6186440677966102
405,CD74_APP,CD74,APP,Mesangial_cells,B cell,True,-52.107051,1.434900e-02,1,False,NaN,NaN,0.32432432432432434
1220,PLXNB2_SEMA4D,PLXNB2,SEMA4D,Mesangial_cells,B cell,True,-4.084814,2.729857e-02,1,True,-2.2159663770794,0.00815986438104258,1
928,CD46_JAG1,CD46,JAG1,B cell,Mesangial_cells,True,-12.690459,6.278330e-08,1,False,NaN,NaN,0.4915254237288136
607,FcRn complex_ALB,"['B2M', 'FCGRT']",ALB,B cell,Mesangial_cells,False,NaN,NaN,"[1.0, 0.472972972972973]",True,-3.15170077783893,0.00163958384977319,1
720,FcRn complex_ALB,"['B2M', 'FCGRT']",ALB,Mesangial_cells,B cell,False,NaN,NaN,"[0.9576271186440678, 0.3389830508474576]",True,-3.15170077783893,0.00163958384977319,1
1300,PODXL_SELL,PODXL,SELL,Mesangial_cells,B cell,True,-92.354564,1.156560e-02,1,False,NaN,NaN,0.6621621621621622


In [11]:
# subset tables for each cell type pair, including self interactions

interactions_downreg_pair = {}

for pair in pairs_coloc_all:
    ct1 = pair[0]
    ct2 = pair[1]
    interactions_downreg_pair[pair] = interactions_downreg[((interactions_downreg['celltype_A'] == ct1) & (interactions_downreg['celltype_B'] == ct2))
                                                | ((interactions_downreg['celltype_B'] == ct1) & (interactions_downreg['celltype_A'] == ct2))]


In [12]:
interactions_downreg_pair[('B cell','Mesangial_cells')]

,interaction,partner_A_genes,partner_B_genes,celltype_A,celltype_B,is_partner_A_DE,logFC_gene_A,adj_pval_gene_A,percent_expr_gene_A,is_partner_B_DE,logFC_gene_B,adj_pval_gene_B,percent_expr_gene_B
585,HLA-F_LILRB2,HLA-F,LILRB2,Mesangial_cells,B cell,False,NaN,NaN,0.1271186440677966,True,9.652284919570539,2.2163431871823598e-10,1
381,ICAM2_aLb2 complex,ICAM2,"['ITGB2', 'ITGAL']",Mesangial_cells,B cell,False,NaN,NaN,0.2033898305084746,True,"[10.1285183950506, 4.76710229557314]","[7.82221790238649e-13, 1.28565836060683e-08]","[1, 1]"
1025,CD55_ADGRE5,CD55,ADGRE5,B cell,Mesangial_cells,False,NaN,NaN,0.5540540540540541,True,4.21690878715005,0.00312233850766636,1
6,COL18A1_a10b1 complex,COL18A1,"['ITGA10', 'ITGB1']",B cell,Mesangial_cells,True,2.6145127218695,0.0454013803960654,1,False,NaN,NaN,"[0.1694915254237288, 0.7288135593220338]"
1091,CD55_ADGRE5,CD55,ADGRE5,Mesangial_cells,B cell,False,NaN,NaN,0.1694915254237288,True,4.21690878715005,0.00312233850766636,1
1466,LTBR_LTB,LTBR,LTB,Mesangial_cells,B cell,False,NaN,NaN,0.1864406779661017,True,1.43943567105272,0.0402619482825343,1
20,COL18A1_a11b1 complex,COL18A1,"['ITGB1', 'ITGA11']",B cell,Mesangial_cells,True,2.6145127218695,0.0454013803960654,1,False,NaN,NaN,"[0.7288135593220338, 0.1016949152542373]"
46,COL18A1_a1b1 complex,COL18A1,"['ITGA1', 'ITGB1']",B cell,Mesangial_cells,True,2.6145127218695,0.0454013803960654,1,False,NaN,NaN,"[0.3898305084745763, 0.7288135593220338]"
957,CXCL12_CXCR4,CXCL12,CXCR4,Mesangial_cells,B cell,True,7.7426278931753885,1.18324783246116e-07,1,False,NaN,NaN,0.945945945945946
125,FN1_a4b1 complex,FN1,"['ITGB1', 'ITGA4']",Mesangial_cells,B cell,True,5.7094563491760715,0.0181158810046262,1,False,NaN,NaN,"[0.32432432432432434, 0.2432432432432433]"


# Producing interactive graphs of cell-cell communication networks dysregulated in abnormal glomeruli

In [13]:
# plotting interactions between 2 particular cell types
cell_types

['B cell',
 'CD8 T cell',
 'DC',
 'Distal_tubules_and_collecting_duct',
 'Endothelium',
 'Loop_of_Henle',
 'MNP',
 'Mast cell',
 'Mesangial_cells',
 'NK cell',
 'NKT cell',
 'Nephron_epithelium',
 'Neutrophil',
 'Podocytes',
 'Private',
 'Proximal_tubular_cells',
 'Th cell']

In [14]:
pairs_coloc_all

[('B cell', 'Mesangial_cells'),
 ('B cell', 'MNP'),
 ('Mesangial_cells', 'Nephron_epithelium'),
 ('Loop_of_Henle', 'Mast cell'),
 ('Distal_tubules_and_collecting_duct', 'NK cell'),
 ('MNP', 'Podocytes'),
 ('Loop_of_Henle', 'Nephron_epithelium'),
 ('DC', 'Podocytes'),
 ('Distal_tubules_and_collecting_duct', 'Nephron_epithelium'),
 ('MNP', 'Private'),
 ('Endothelium', 'Mesangial_cells'),
 ('Endothelium', 'MNP'),
 ('DC', 'Private'),
 ('B cell', 'CD8 T cell'),
 ('B cell', 'Loop_of_Henle'),
 ('B cell', 'DC'),
 ('B cell', 'Nephron_epithelium'),
 ('CD8 T cell', 'Endothelium'),
 ('Mast cell', 'Private'),
 ('Loop_of_Henle', 'Neutrophil'),
 ('NK cell', 'Neutrophil'),
 ('DC', 'MNP'),
 ('Distal_tubules_and_collecting_duct', 'Mesangial_cells'),
 ('Nephron_epithelium', 'Neutrophil'),
 ('Endothelium', 'Loop_of_Henle'),
 ('B cell', 'Neutrophil'),
 ('Endothelium', 'Nephron_epithelium'),
 ('MNP', 'NK cell'),
 ('DC', 'NK cell'),
 ('MNP', 'Proximal_tubular_cells'),
 ('Distal_tubules_and_collecting_duct', 

In [17]:
# upregulated interactions
n_cell_types = len(cell_types)

nx_graph = nx.null_graph()
nx_graph

for ct in cell_types:
    nx_graph.add_node(ct, size=5, title=ct)

# add edges from pairs_coloc_all
for pair in pairs_coloc_all:
    ct_A = pair[0]
    ct_B = pair[1]
    # number of upregulated interactions
    n_inter_upreg = interactions_upreg_pair[pair].shape[0]
    nx_graph.add_edge(ct_A, ct_B, weight=5,
                      title = str(n_inter_upreg) + ' upregultead interactions: ' + str(list(interactions_upreg_pair[pair]['interaction'])))
    
#nx_graph.add_node(25, size=25, label='lonely', title='lonely node', group=3)
nt = Network('750px', '1400px')
# populates the nodes and edges data structures
nt.from_nx(nx_graph)
nt.show('nx_upreg.html')

In [18]:
# downregulated interactions
n_cell_types = len(cell_types)

nx_graph = nx.null_graph()
nx_graph

for ct in cell_types:
    nx_graph.add_node(ct, size=5, title=ct)

# add edges from pairs_coloc_all
for pair in pairs_coloc_all:
    ct_A = pair[0]
    ct_B = pair[1]
    # number of upregulated interactions
    n_inter_downreg = interactions_downreg_pair[pair].shape[0]
    nx_graph.add_edge(ct_A, ct_B, weight=5,
                      title = str(n_inter_downreg) + ' downregultead interactions: ' + str(list(interactions_downreg_pair[pair]['interaction'])))
    
#nx_graph.add_node(25, size=25, label='lonely', title='lonely node', group=3)
nt = Network('750px', '1400px')
# populates the nodes and edges data structures
nt.from_nx(nx_graph)
nt.show('nx_downreg.html')

# You are now welcome to open generated interactive `nx_upreg.html` and `nx_downreg.html` HTML files to explore interactions dyrsegulated in abnormal vs healthy glomeruli